In [5]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
mpl.rc('font', family='Malgun Gothic')

In [6]:
df_member = pd.read_csv('Member_Data.csv', encoding='EUC-KR')
df_product = pd.read_csv('Product_Data.csv')
df_sales = pd.read_csv('Sales_Data05.csv')

In [16]:
df_member.head(5)

,Unnamed: 0,회원번호,회원상태,성별,나이,등록카드,결혼,구독여부,주소지,세부주소지
0,0,18764160,정상회원,여,68,농협중앙회,기혼,False,서울특별시,성동구
1,1,18792000,정상회원,남,83,연결앱결제,NaN,False,강원도,강릉시
2,2,18942336,정상회원,여,39,신한은행,기혼,False,인천광역시,중구
3,3,18949760,정상회원,여,73,기업은행,NaN,NaN,강원도,홍천군
4,4,19391488,정상회원,여,52,연결앱결제,기혼,False,대전광역시,중구


In [18]:
df_product.head(5)

,제품번호,물품명,물품대분류,물품중분류,상품중량
0,100021V2_0,2단무늬컵,식기/편백,자기,1p
1,100022V2_1,7곡딸기롤과자,과자,스낵,100g/10개입
2,100023V2_2,7곡참식,식사대용,선식/생식,700g
3,100024V2_3,가리비,생물수산,패류/갑각류,1.5kg
4,100025V2_3,가리비,생물수산,패류/갑각류,1kg


In [20]:
df_sales.head(5)

,회원번호,회원상태,구매수량,구매금액,주문일시,배송시작일,배송완료일,사용 적립금,사용 포인트 네이버,주문취소여부,주문시간,제품번호,주문일시_dt,주문일시_year,주문일시_month,주문일시_day
0,1032097472,정상회원,1.0,7083,2021-01-02,NaN,NaN,0,0,주문취소,오후 12:60,100021783V2_1337,2021-01-02,2021,1,2
1,1032097472,정상회원,1.0,29865,2021-01-02,2021-01-02,2021-01-02,0,0,NaN,오후 12:60,100022137V2_1606,2021-01-02,2021,1,2
2,1032097472,정상회원,1.0,23164,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,100021452V2_1113,2021-01-02,2021,1,2
3,369152832,정상회원,1.0,16655,2021-01-02,2021-01-02,2021-01-03,0,0,NaN,오후 12:60,10002931V2_708,2021-01-02,2021,1,2
4,1032097472,정상회원,1.0,8423,2021-01-02,NaN,NaN,0,0,주문취소,오후 12:60,100022085V2_1559,2021-01-02,2021,1,2


# 고객 이탈률
- (이탈 고객수) / (전체 고객수) * 100

In [27]:
# "회원상태" 하나 null
df_member['회원상태'].value_counts()

회원상태
정상회원     12380
탈퇴         156
탈퇴처리중        2
탈퇴신청         1
Name: count, dtype: int64

In [7]:
# '정상회원'이 아닐 경우 '고객 이탈'로 가정
cond_leave = df_member['회원상태'] != "정상회원"

# 이탈 고객 수
leave_num = len(df_member.loc[cond_leave].index)

# 전체 고객 수
member_num = len(df_member['회원상태'])

# 고객 이탈률 계산
customer_leave_rate = leave_num / member_num * 100
print(f"고객 이탈률은 {customer_leave_rate}% 이다.")

고객 이탈률은 1.2759170653907497% 이다.


# n개월 이상 구매 유지 고객비율
- (n개월 이상 구매 유지 고객 수) / (전체 고객 수) * 100
- 처음 주문과 마지막 주문으로 n개월 이상 구매를 유지하는지 정의함.
- 두 달에 걸쳐 구매 이력 O = 2개월 이상 구매 유지

In [148]:
# 주문 취소 여부 고려할 필요 있음.
# 회원가입만 하고 구매하지 않은 고객이 있을 가능성, df_member와 병합 후 계산할 필요

# '주문일시' datetime 전환
df_sales['주문일시_dt'] = pd.to_datetime(df_sales['주문일시'])
df_sales['주문일시_year'] = df_sales['주문일시_dt'].dt.year
df_sales['주문일시_month'] = df_sales['주문일시_dt'].dt.month
df_sales['주문일시_day'] = df_sales['주문일시_dt'].dt.day

# 회원별 처음 주문 날짜와 마지막 주문 날짜를 df_orderdate에 저장
# 연도는 모두 2021로 동일하므로 고려하지 않음.
df_orderdate = df_sales.sort_values(by=['회원번호', '주문일시_month']).pivot_table(index = '회원번호',
                                                                           values = '주문일시_month', aggfunc = ['min', 'max']).reset_index()
# 몇개월 동안 구매 유지 중인지 계산하여 'n개월구매유지'에 저장
df_orderdate['n개월구매유지'] = ( df_orderdate['max']['주문일시_month'] - df_orderdate['min']['주문일시_month'] )

# 입력
n = int(input())

# n개월 이상 구매 유지 중인 고객 수
more_n_cond = df_orderdate['n개월구매유지'] >= n
keep_customer_num = len(df_orderdate.loc[more_n_cond].index)

# 전체 고객 수
total_customer_num = len(df_member.index)

# n개월 이상 구매 유지 고객 비율
customer_keep_rate = keep_customer_num / total_customer_num * 100
print(f"{n}개월 이상 구매 유지 고객비율은 {customer_keep_rate}% 이다.")

 1


1개월 이상 구매 유지 고객비율은 74.58532695374801% 이다.


In [141]:
# 확인용
#df_sales['연'].value_counts()
#df_orderdate['n개월구매유지'].value_counts()

# 배송지연율
- (배송 지연 수) / (전체 배송 수) * 100 
- 2일 이상 소모 시 배송 지연으로 가정.

In [73]:
# '배송시작일' datetime 전환
df_sales['배송시작일_dt'] = pd.to_datetime(df_sales['배송시작일'])

# '배송완료일' datetime 전환
df_sales['배송완료일_dt'] = pd.to_datetime(df_sales['배송완료일'])

# '배송소모일' 구하기
df_sales['배송소모일'] = (df_sales['배송완료일_dt'] - df_sales['배송시작일_dt']).dt.days

# 배송 지연 수
delivery_late_cond = df_sales['배송소모일'] >= 2
delivery_late_num = len(df_sales.loc[delivery_late_cond].index)

# 전체 배송 수
delivery_cond = pd.notna(df_sales['배송시작일'])
total_delivery_num = len(df_sales.loc[delivery_cond].index)

# 배송지연율
delivery_late_rate = delivery_late_num / total_delivery_num * 100
print(f"배송지연율은 {delivery_late_rate}% 이다.")

배송지연율은 15.226769321511712% 이다.


In [79]:
# 확인용
#df_sales['배송소모일'].value_counts()

# 정기구독률
- 정기구독률 = (구독 회원수) / (전체 회원수) * 100

In [81]:
# 구독X = NaN(결측값), False  /  구독O =  True 가정.
df_member['구독여부'].value_counts()

구독여부
False    8441
True     1753
Name: count, dtype: int64

In [150]:
# 구독 회원 수
subscription_cond = df_member['구독여부'] == True
subscription_customer = len(df_member.loc[subscription_cond].index)

# 전체 회원 수
total_customer = len(df_member.index)

# 정기 구독률
subscription_rate = subscription_customer / total_customer * 100
print(f"정기구독률은 {subscription_rate}% 이다.")

정기구독률은 13.9792663476874% 이다.


# 유령고객 비율
- 유령고객 정의 : 가장 최신 주문 건 기준 n개월 간 주문이 없는 고객
- n입력을 '월'이 아닌 '일'로 받음. 추후 수정 필요.

In [158]:
# '주문일시' datetime 전환
df_sales['주문일시_dt'] = pd.to_datetime(df_sales['주문일시'])

# 가장 최근 주문 날짜
last_order_dt = df_sales['주문일시_dt'].max()

# 회원번호 별 최근 주문 일시 데이터프레임
df_customer_last_order = df_sales.pivot_table(index = '회원번호', values = '주문일시_dt', aggfunc = 'max').reset_index()

# '주문기간' : 주문하지 않은 기간
df_customer_last_order['주문기간'] = (last_order_dt - df_customer_last_order['주문일시_dt']).dt.days

# 입력
n = int(input("몇일 간 주문이 없으면 유령고객인지 입력하시오 :"))

# 유령 고객 수
ghost_cond = df_customer_last_order['주문기간'] >= n
ghost_customer = len(df_customer_last_order.loc[ghost_cond].index)

# 전체 고객 수
total_customer = len(df_member.index)

# 유령고객 비율
ghost_customer_rate = ghost_customer / total_customer * 100
print(f"유령고객 비율은 {ghost_customer_rate}% 이다.")

몇일 간 주문이 없으면 유령고객인지 입력하시오 : 60


유령고객 비율은 37.60765550239235% 이다.


In [160]:
# 확인용
#df_customer_last_order['주문기간'].value_counts()